In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session~
#a lward ifires djilali d ahwawi ihemmey adihewes

/kaggle/input/are-sd-2025-classification-diagnostique-d-arbres/benchmark.csv
/kaggle/input/are-sd-2025-classification-diagnostique-d-arbres/prev.csv
/kaggle/input/are-sd-2025-classification-diagnostique-d-arbres/train.csv


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
data = pd.read_csv('/kaggle/input/are-sd-2025-classification-diagnostique-d-arbres/train.csv', index_col='ID_ARBRE')
prev_data = pd.read_csv('/kaggle/input/are-sd-2025-classification-diagnostique-d-arbres/prev.csv', index_col='ID_ARBRE')
prev_data.head(3)


,quartier,site,genre_arbre,situation,type_sol,surf_permeable,classe_age,hauteur,classe_hauteur,diametre,...,fissure_houppier,ecorce_incluse_houppier,bois_mort_houppier,plaie_houppier,observation_houppier,esperance_maintien,contrainte,Long,Lat,Usage
ID_ARBRE,,,,,,,,,,,,,,,,,,,,,
559,Quartier 2 - Alsace - Pereire,Avenue du Maréchal Foch,Tilia,Alignement,MA,4.0,A,800,H2,44.563384,...,HPF,Non,HPBM,HPLNC,Branches cassées,2.0,Non,2.084,48.904,Private
560,Quartier 4 - Rotondes - St Léger,Avenue Saint-Fiacre,Carpinus,Alignement,Gr,1.0,JA,800,H2,38.197186,...,HPF,Non,HPBM,HPLS,0,1.0,Non,2.072,48.895,Private
561,Quartier 2 - Alsace - Pereire,Avenue du Maréchal Foch,Tilia,Alignement,MA,4.0,A,800,H2,63.661977,...,HPF,Non,HPBM,HPLS,0,2.0,Non,2.083,48.904,Private


****ALGO DES KNN****

In [4]:
import numpy as np

def ARE(prev,true,v = 'classification_diagnostic'):
    """
    Calcul le taux de réussite entre prev et true
    Paramètres :
    -----------
    prev : prévisions faites par notre modèle
    true : résultat attendu dans nos données tests
    v    : Nom de la variable de sortie de notre modèle et des données tests
    """
    return 100*(prev[v].reset_index(drop=True) == true[v].reset_index(drop=True)).mean()


def label_encoder(feature_vec):
    """
    Encode des étiquettes extraits d'une colonne des données d'entraînement
    L'encodage consiste à remplacer les labels (chaînes de caractères) par des entiers 
    afin de pouvoir calculer une distance entre deux données
    """
    _, encoded_labels = np.unique(feature_vec, return_inverse=True)
    return encoded_labels # Retourne les indices des modalités

def euclidian_distance(x_train, x_test):
    """
    Calcule la distance euclidienne entre x_train et x_test.

    La distance entre une donnée d'entraînement et une donnée test est défine comme la racine carrée de :
                sum( (x_train[i] - x_test[i])^2 )
                 i
    Cette somme peut être exprimée sous forme vectorielle :

    sum( (x_train[i] - x_test[i])^2 ) = < x_train - x_test | x_train - x_test> 
     i
                                      = <x_train | x_train> + <x_test | x_test> - 2.<x_train | x_test>
    """
    sqr_dist = -2*x_train@x_test.T + np.sum(x_test**2,axis=1) + np.sum(x_train**2,axis=1)[:,np.newaxis]
    sqr_dist[sqr_dist<0] = 0.
    # Retour de la distance euclidienne
    return np.sqrt(sqr_dist)

In [5]:
def normalise(vec : pd.DataFrame):
    """Normalise les données de vec """
    vmax = np.max(vec)
    vmin = np.min(vec)
    #pour empecher une division par 0
    if vmax == vmin:
        return np.zeros(len(vec))
    return (vec-vmin)/(vmax-vmin)


In [6]:
def knn(train_data : pd.DataFrame, prev_data : pd.DataFrame, vvar : list, k : int, normalise_f):
    """Prédit les classes de prev_data avec knn
    *train_data : données d'entrainement
    *prev_data : données à prédire
    *vvar : les variables utilisées pour calculer les distances
    *k : le nombre de voisins pris en compte
    *normalise_f : fonction de normalisation des données"""

    #Normalisation et labelisation des données de train_data
    vars_norm = []
    for var in vvar:
        if type( train_data.iloc[0][var] ) == str:
            vars_norm.append( normalise_f( label_encoder(train_data[var]) ) )
        else:
            vars_norm.append( normalise_f(train_data[var]) )
    train_data_n = np.vstack((vars_norm)).T

    #Normalisation et labélisation des données de prev_data
    vars_norm = []
    for var_p in vvar:
        if type( prev_data.iloc[0][var_p] ) == str:
            vars_norm.append( normalise_f( label_encoder(prev_data[var_p]) ) )
        else:
            vars_norm.append( normalise_f(prev_data[var_p]) )
    prev_data_n = np.vstack((vars_norm)).T
    dist = euclidian_distance(prev_data_n, train_data_n)
    k_plus_proches = np.argsort(dist, 1)  
    votants = []
    for i in range(len(prev_data)):
        votants.append([train_data.iloc[j]["classification_diagnostic"] for j in k_plus_proches[i, :k] ])
    v=[]
    cpt=[]
    for i in range(len(prev_data)):
        v_tmp,cpt_tmp = np.unique(votants[i], return_counts  = True)
        #v_tmp contient : np.unique(votants[i]) et cpt_tmp contient le nombre d'occurence de chaque classes. 
        v.append(v_tmp)
        cpt.append(cpt_tmp)
    return [v[i][np.argmax(cpt[i])] for i in range(len(prev_data)) ]

***Entrainement du modèle***

In [7]:
def validation_croisée_knn(data: pd.DataFrame, predi_fonc, vvars: np.array ,k: int, normalise_f, n_iter: int):
    """Renvoie le taux de réussite de la prediction du niveau de blessure des arbres par la fonction predi_f 
    testé sur data_test.
    *data : données à utiliser
    *predi_fonc : fonciton qui prédit (selon knn)
    *vvars : les variables utilisées pour calculer les distances
    *normalise_f : fonction de normalisation des données
    n_iter : nombre de fois ou predi_fonc sera testé 
    """
    long = len(data) // 5
    res = []
    for i in range(n_iter):
        v_rd = np.random.permutation(data.index)
        data_test = data[data.index.isin(v_rd[:long])]
        data_train = data[data.index.isin(v_rd[long:])]
        prediction = predi_fonc(data_train, data_test, vvars, k, normalise_f)
        realite = data_test["classification_diagnostic"].to_numpy()
        res.append(np.mean(prediction == realite) * 100)
    return np.mean(res)


In [8]:
""" PREND TROP DE TEMPS A S'EXECUTER 
from tqdm import tqdm

dico1k = {"vars" : [], "score" : [], "k" : []}

for _ in tqdm(range(500)):
    print(dico1k["vars"])
    rd = np.random.permutation(len(vars_tout))
    for k in range(5, 17):
        res1 = validation_croisée_knn(data, knn_tout,[vars_tout[i] for i in rd[:13]] , k, normalise, 50)
        dico1k["vars"].append([vars_tout[i] for i in rd[:13] ])
        dico1k["score"].append(res1)
        dico1k["k"].append(k)"""

' PREND TROP DE TEMPS A S\'EXECUTER \nfrom tqdm import tqdm\n\ndico1k = {"vars" : [], "score" : [], "k" : []}\n\nfor _ in tqdm(range(500)):\n    print(dico1k["vars"])\n    rd = np.random.permutation(len(vars_tout))\n    for k in range(5, 17):\n        res1 = validation_croisée_knn(data, knn_tout,[vars_tout[i] for i in rd[:13]] , k, normalise, 50)\n        dico1k["vars"].append([vars_tout[i] for i in rd[:13] ])\n        dico1k["score"].append(res1)\n        dico1k["k"].append(k)'

***Variables et paramètres optimaux***

In [9]:
vvars = ['champignon_houppier',
  'plaie_collet',
  'ecorce_incluse_houppier',
  'fissure_houppier',
  'surf_permeable',
  'genre_arbre',
  'bois_mort_houppier',
  'hauteur',
  'classe_hauteur',
  'quartier',
  'vigueur_pousse',
  'classe_age']
v_vars = ['vigueur_pousse',
 'quartier',
 'ecorce_incluse_houppier',
 'observation_tronc',
 'port_arbre',
 'classe_age',
 'insecte_tronc',
 'champignon_collet',
 'canisse_arbre',
 'plaie_houppier',
 'champignon_tronc',
 'surf_permeable',
 'circonference (en cm)',
 'fissure_tronc',
 'classe_hauteur',
 'insecte_collet',
 'plaie_collet']

***Fichier à soumettre***

In [10]:
predi_knn = knn(data, prev_data, v_vars, 5, normalise)
index_knn = np.array([i for i in prev_data.index])
v_res_knn = np.vstack((index_knn, predi_knn)).T
df_knn = pd.DataFrame(v_res_knn, columns=[['ID_ARBRE', 'classification_diagnostic']])
df_knn.to_csv("vpredi_knn.csv", index = False)
df_knn 

,ID_ARBRE,classification_diagnostic
0,559,C2
1,560,C2
2,561,C2
3,562,C1
4,563,C3
...,...,...
132,694,C2
133,695,C2
134,696,C2
135,697,C1
